book: MİRZƏ FƏTƏLİ AXUNDOV - SƏRGÜZƏŞTİ-MƏRDİ-XƏSİS (Hacı Qara) 

pdf link: https://www.azeri.org/Azeri/az_latin/latin_lit/az_literature/drama/mirza_fatali_akhundov/akhundov_az/akhundov_hajigara.pdf

In [1]:
import pandas as pd
import numpy as np

In [2]:
original = pd.read_csv("original.csv")
original.head()

,id,text
0,1,"Pərvərdigara, bu necə əsrdir?"
1,2,Bu necə zəmanədir?
2,3,"Nə at çapmağın qiyməti var, nə tüfəng atmağın ..."
3,4,"Sabahdan axşamadək, axşamdan sabahadək arvad k..."
4,5,"Dövlət dəxi haradan olsun, pul haradan olsun!?"


In [3]:
modern = pd.read_csv("modern.csv")
modern.head()

,id,text
0,1,"Pərvərdigara, bu nə dövrdür?"
1,2,Bu nə zamandır belə?
2,3,"Nə at çapmağın dəyəri qalıb, nə də tüfəng atma..."
3,4,"Səhərdən axşama, axşamdan səhərə qədər qadın k..."
4,5,"Dövlət haradan olsun, pul haradan gəlsin?"


In [4]:
df = pd.DataFrame({
    "id": original["id"],
    "original": original["text"],
    "modern": modern["text"]
})

df.head()

,id,original,modern
0,1,"Pərvərdigara, bu necə əsrdir?","Pərvərdigara, bu nə dövrdür?"
1,2,Bu necə zəmanədir?,Bu nə zamandır belə?
2,3,"Nə at çapmağın qiyməti var, nə tüfəng atmağın ...","Nə at çapmağın dəyəri qalıb, nə də tüfəng atma..."
3,4,"Sabahdan axşamadək, axşamdan sabahadək arvad k...","Səhərdən axşama, axşamdan səhərə qədər qadın k..."
4,5,"Dövlət dəxi haradan olsun, pul haradan olsun!?","Dövlət haradan olsun, pul haradan gəlsin?"


In [5]:
df["sentiment"] = ""
df.to_csv("sent_pairs.csv", index=False, encoding="UTF-8")
df.head()

,id,original,modern,sentiment
0,1,"Pərvərdigara, bu necə əsrdir?","Pərvərdigara, bu nə dövrdür?",
1,2,Bu necə zəmanədir?,Bu nə zamandır belə?,
2,3,"Nə at çapmağın qiyməti var, nə tüfəng atmağın ...","Nə at çapmağın dəyəri qalıb, nə də tüfəng atma...",
3,4,"Sabahdan axşamadək, axşamdan sabahadək arvad k...","Səhərdən axşama, axşamdan səhərə qədər qadın k...",
4,5,"Dövlət dəxi haradan olsun, pul haradan olsun!?","Dövlət haradan olsun, pul haradan gəlsin?",


In [6]:
df2 = pd.read_csv("pairs.csv") #sentiment part was updated
df2.head()

,id,original,modern,sentiment
0,1,"Pərvərdigara, bu necə əsrdir?","Pərvərdigara, bu nə dövrdür?",negative
1,2,Bu necə zəmanədir?,Bu nə zamandır belə?,negative
2,3,"Nə at çapmağın qiyməti var, nə tüfəng atmağın ...","Nə at çapmağın dəyəri qalıb, nə də tüfəng atma...",negative
3,4,"Sabahdan axşamadək, axşamdan sabahadək arvad k...","Səhərdən axşama, axşamdan səhərə qədər qadın k...",negative
4,5,"Dövlət dəxi haradan olsun, pul haradan olsun!?","Dövlət haradan olsun, pul haradan gəlsin?",negative


In [7]:
label2id = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}
id2label = {v: k for k, v in label2id.items()}

df2["sentiment_id"] = df2["sentiment"].map(label2id)
df2.head()

,id,original,modern,sentiment,sentiment_id
0,1,"Pərvərdigara, bu necə əsrdir?","Pərvərdigara, bu nə dövrdür?",negative,0
1,2,Bu necə zəmanədir?,Bu nə zamandır belə?,negative,0
2,3,"Nə at çapmağın qiyməti var, nə tüfəng atmağın ...","Nə at çapmağın dəyəri qalıb, nə də tüfəng atma...",negative,0
3,4,"Sabahdan axşamadək, axşamdan sabahadək arvad k...","Səhərdən axşama, axşamdan səhərə qədər qadın k...",negative,0
4,5,"Dövlət dəxi haradan olsun, pul haradan olsun!?","Dövlət haradan olsun, pul haradan gəlsin?",negative,0


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### model 1: XLM-RoBERTa:

In [9]:
xlm_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
xlm_tokenizer = AutoTokenizer.from_pretrained(xlm_name)
xlm_model = AutoModelForSequenceClassification.from_pretrained(xlm_name)

In [10]:
def predict_xlm(texts):
    inputs = xlm_tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = xlm_model(**inputs)
    preds = torch.argmax(outputs.logits, dim=1)
    return preds.numpy()


### model 2: Multilingual BERT:

In [11]:
bert_name = "nlptown/bert-base-multilingual-uncased-sentiment"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_name)

In [12]:
def predict_bert(texts):
    inputs = bert_tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = bert_model(**inputs)

    stars = torch.argmax(outputs.logits, dim=1) + 1

    mapped = []
    for s in stars:
        if s <= 2:
            mapped.append(0)   # negative
        elif s == 3:
            mapped.append(1)   # neutral
        else:
            mapped.append(2)   # positive

    return mapped


### prediction & calculating f1 / recall / precision

In [13]:
texts = df2["modern"].tolist()

df2["xlm_pred"] = predict_xlm(texts)
df2["bert_pred"] = predict_bert(texts)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
from sklearn.metrics import classification_report

In [15]:
print("XLM-RoBERTa results:")
print(
    classification_report(
        df2["sentiment_id"],
        df2["xlm_pred"],
        target_names=["negative", "neutral", "positive"]
    )
)

XLM-RoBERTa results:
              precision    recall  f1-score   support

    negative       0.68      0.47      0.56        32
     neutral       0.30      0.57      0.39        14
    positive       0.50      0.20      0.29         5

    accuracy                           0.47        51
   macro avg       0.49      0.41      0.41        51
weighted avg       0.56      0.47      0.48        51



In [16]:
print("mBERT results:")
print(
    classification_report(
        df2["sentiment_id"],
        df2["bert_pred"],
        target_names=["negative", "neutral", "positive"]
    )
)

mBERT results:
              precision    recall  f1-score   support

    negative       0.59      0.62      0.61        32
     neutral       0.18      0.14      0.16        14
    positive       0.17      0.20      0.18         5

    accuracy                           0.45        51
   macro avg       0.31      0.32      0.32        51
weighted avg       0.44      0.45      0.44        51



### similarity

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel

In [18]:
def get_embeddings(texts, tokenizer, model):
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()

In [19]:
xlm_name = "xlm-roberta-base"
xlm_tokenizer = AutoTokenizer.from_pretrained(xlm_name)
xlm_model = AutoModel.from_pretrained(xlm_name)

In [20]:
bert_name = "bert-base-multilingual-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_name)
bert_model = AutoModel.from_pretrained(bert_name)

In [21]:
orig_texts = df2["original"].tolist()
modern_texts = df2["modern"].tolist()

# XLM-R
orig_emb_xlm = get_embeddings(orig_texts, xlm_tokenizer, xlm_model)
modern_emb_xlm = get_embeddings(modern_texts, xlm_tokenizer, xlm_model)

# mBERT
orig_emb_bert = get_embeddings(orig_texts, bert_tokenizer, bert_model)
modern_emb_bert = get_embeddings(modern_texts, bert_tokenizer, bert_model)


In [22]:
similarity_xlm = cosine_similarity(orig_emb_xlm, modern_emb_xlm).diagonal()
similarity_bert = cosine_similarity(orig_emb_bert, modern_emb_bert).diagonal()

In [23]:
df2["sim_xlm"] = similarity_xlm
df2["sim_bert"] = similarity_bert

print("Average similarity (XLM-R):", df2["sim_xlm"].mean())
print("Average similarity (mBERT):", df2["sim_bert"].mean())

Average similarity (XLM-R): 0.97820836
Average similarity (mBERT): 0.86931986


### final result

In [24]:
df2.to_csv("final_results.csv", index=False, encoding="utf-8")